Data preprocessing:

The Data preprosessing contains 4 parts: 

Part 1: Load, filter, and missing values
We use LendingClub’s loan.csv to predict recovery ratio (total_pymnt / loan_amnt). We keep only closed loans (Fully Paid, Charged Off, Default), drop columns with more than 40% missing, and impute the rest: median for numeric, most-frequent for categorical. 

Part 2: Data split
We sort by issue date and split 70% train / 15% val / 15% test so the model never uses future data. X only has features; target and outcome columns are removed to avoid leakage. 

Part 3: Outliers and feature engineering
We do not remove outliers; StandardScaler and tree models handle them. We clean term (36 or 60), emp_length (e.g. "10+ years" → 10), and ordinal-encode grade/sub_grade (A=1…G=7). We add issue_year and issue_month from the issue date. 

Part 4: ColumnTransformer and one-hot encoding
Using ColumnTransformer: numeric columns get median impute → StandardScaler; categorical columns get most-frequent impute → OneHotEncoder with handle_unknown="ignore". The preprocessor is fit on train and used to transform train, val, and test.